In [ ]:
from appgeopy import *
from my_packages import *

In [ ]:
# Function to detect peaks and troughs
def detect_peaks_troughs(signal, time_index):
    # Detect peaks
    peaks, properties_peaks = scipy.signal.find_peaks(signal, prominence=True, distance=10)
    peak_times = time_index[peaks]

    # Detect troughs (by inverting the signal)
    troughs, properties_troughs = scipy.signal.find_peaks(-signal, prominence=True, distance=10)
    trough_times = time_index[troughs]

    return peaks, properties_peaks, peak_times, troughs, properties_troughs, trough_times


# Function to plot peaks and annotate them
def plot_peaks(ax, signal, time_index, peak_times, peaks, properties_peaks, scaling_factor):
    ax.plot(time_index, signal, label="Signal", color="gray")
    ax.plot(peak_times, signal[peaks], "go", label="Peaks", ms=7, alpha=0.6)

    # Highlight prominence for each peak
    for i in range(len(peaks)):
        ax.vlines(
            x=peak_times[i],
            ymin=signal[peaks[i]] - properties_peaks["prominences"][i],
            ymax=signal[peaks[i]],
            color="purple",
            linestyle="--",
            label="Prominence" if i == 0 else "",
        )

    # Annotate peaks with their indexes
    for i, peak in enumerate(peaks):
        ax.annotate(
            f"{peak}",
            xy=(peak_times[i], signal[peak]),
            xytext=(0, 10),
            textcoords="offset points",
            ha="center",
            va="bottom",
            rotation="vertical",
            color="black",
            fontsize=9 * scaling_factor,
        )

    # Add horizontal lines between consecutive peaks
    for i in range(1, len(peaks)):
        ax.hlines(
            y=min(signal[peaks[i]], signal[peaks[i - 1]]) - 0.1,
            xmin=peak_times[i - 1],
            xmax=peak_times[i],
            color="blue",
            linestyle="--",
            label="Distance" if i == 1 else "",
            alpha=0.5
        )


# Function to plot troughs and annotate them
def plot_troughs(ax, signal, time_index, trough_times, troughs, properties_troughs, scaling_factor):
    ax.plot(time_index, signal, label="Signal", color="gray")
    ax.plot(trough_times, signal[troughs], "ro", label="Troughs", ms=7, alpha=0.6)

    # Highlight prominence for each trough
    for i in range(len(troughs)):
        ax.vlines(
            x=trough_times[i],
            ymin=signal[troughs[i]] + properties_troughs["prominences"][i],
            ymax=signal[troughs[i]],
            color="orange",
            linestyle="--",
            label="Prominence" if i == 0 else "",
        )

    # Annotate troughs with their indexes
    for i, trough in enumerate(troughs):
        ax.annotate(
            f"{trough}",
            xy=(trough_times[i], signal[trough]),
            xytext=(0, -12),
            textcoords="offset points",
            ha="center",
            va="top",
            rotation="vertical",
            color="black",
            fontsize=9 * scaling_factor,
        )

    # Add horizontal lines between consecutive troughs
    for i in range(1, len(troughs)):
        ax.hlines(
            y=max(signal[troughs[i]], signal[troughs[i - 1]]) + 0.1,
            xmin=trough_times[i - 1],
            xmax=trough_times[i],
            color="cyan",
            linestyle="--",
            label="Distance" if i == 1 else "",
            alpha=0.5
        )


def plot_combined(ax, signal, time_index, peak_times, peaks, trough_times, troughs, scaling_factor):
    ax.plot(time_index, signal, label="Signal", color="gray")

    # Plot peaks
    ax.plot(peak_times, signal[peaks], "go", label="Peaks", ms=7, alpha=0.6)
    # Annotate peaks with their indexes
    for i, peak in enumerate(peaks):
        ax.annotate(
            f"{peak}",
            xy=(peak_times[i], signal[peak]),
            xytext=(0, 10),
            textcoords="offset points",
            ha="center",
            va="bottom",
            rotation="vertical",
            color="black",
            fontsize=9 * scaling_factor,
        )

    # Plot troughs
    ax.plot(trough_times, signal[troughs], "ro", label="Troughs", ms=7, alpha=0.6)
    # Annotate troughs with their indexes
    for i, trough in enumerate(troughs):
        ax.annotate(
            f"{trough}",
            xy=(trough_times[i], signal[trough]),
            xytext=(0, -12),
            textcoords="offset points",
            ha="center",
            va="top",
            rotation="vertical",
            color="black",
            fontsize=9 * scaling_factor,
        )


# Function to configure and finalize the plot
def configure_plot(ax, title, scaling_factor):
    visualize.configure_axis(
        ax=ax,
        xlabel="",
        ylabel="Groundwater Levels (m)",
        title=title,
        scaling_factor=scaling_factor,
    )
    visualize.configure_legend(ax=ax, scaling_factor=0.5, frameon=False, fontsize_base=12)
    visualize.configure_datetime_ticks(ax=ax, axis="x", major_interval=12, minor_interval=6, date_format="%Y")


# Main function to handle the plotting, now with a third subplot
def plot_peaks_troughs(signal, time_index, scaling_factor):
    # Detect peaks and troughs
    peaks, properties_peaks, peak_times, troughs, properties_troughs, trough_times = detect_peaks_troughs(signal, time_index)

    # Set up figure with three subplots
    fig_width, fig_height = (16.5, 11.7)
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(fig_width, fig_height), sharex=True)

    # Plot peaks (pass time_index as an argument)
    plot_peaks(ax1, signal, time_index, peak_times, peaks, properties_peaks, scaling_factor)
    
    # Plot troughs (pass time_index as an argument)
    plot_troughs(ax2, signal, time_index, trough_times, troughs, properties_troughs, scaling_factor)

    # Plot combined peaks and troughs in ax3
    plot_combined(ax3, signal, time_index, peak_times, peaks, trough_times, troughs, scaling_factor)

    # Configure all axes
    configure_plot(ax1, title="", scaling_factor=scaling_factor)
    configure_plot(ax2, title="", scaling_factor=scaling_factor)
    configure_plot(ax3, title="", scaling_factor=scaling_factor)

    # Final adjustments and display
    fig.suptitle(f"{station} {wellcode}", fontsize=20, fontweight="bold", y=0.925)
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    # Apply rotation to the x-tick labels on the shared axis (ax3)
    plt.setp(ax3.get_xticklabels(), rotation=90, ha="center")

    return fig

In [ ]:
# _______________________________________________________________________
# Load HDF5 File and Extract Dataset Information
hdf5_fpath = r"20240903_GWL_CRFP.h5"

with h5py.File(hdf5_fpath, "r") as hdf5_file:
    # Extract existing data and available datasets
    existing_data_dict = h5pytools.hdf5_to_data_dict(hdf5_file)
    available_datasets = h5pytools.list_datasets(hdf5_file)

    # Extract the 'date' array and convert to a datetime index
    datetime_array = pd.to_datetime(existing_data_dict["date"], format="%Y%m%d")

# _______________________________________________________________________
# Get list of stations
stations = sorted(set([elem.split("/")[0] for elem in available_datasets if "date" not in elem]))

In [ ]:
# station = "BOZI"
for station in tqdm(stations[:]):

    # Extract the station's data from the dictionary
    station_data = existing_data_dict[station]

    # Filter wellcodes that contain exactly 3 items (indicating wellcode structure)
    wellcodes = [elem for elem, val in station_data.items() if isinstance(val, dict) and len(val) == 3]

    for wellcode in tqdm(wellcodes, leave=False):

        # Extract and process time series data
        model_gwl_arr = station_data[wellcode]["measure"]["model"]
        model_gwl_series = pd.Series(data=model_gwl_arr, index=datetime_array)

        # Optional: Save the plot to a file
        output_savename = f"{station}_{wellcode}"
        savepath = f"temp5\\{output_savename}.png"

        # Call the main plotting function
        outputfig = plot_peaks_troughs(
            signal=model_gwl_series.values, time_index=model_gwl_series.index, scaling_factor=1.2
        )
        visualize.save_figure(outputfig, savepath)
        plt.close()